<a href="https://colab.research.google.com/github/yoonhero/nanoGPT/blob/master/gpt_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-03-12 01:50:02--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2023-03-12 01:50:02 (19.7 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [3]:
len(text)

1115394

In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# string to index
stoi = {char:i for i, char in enumerate(chars)}
itos = {i:char for i, char in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

print(encode("hello"))
print(decode(encode("wow")))

[46, 43, 50, 50, 53]
wow


In [7]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [8]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'context: {context} target: {target}')

context: tensor([18]) target: 47
context: tensor([18, 47]) target: 56
context: tensor([18, 47, 56]) target: 57
context: tensor([18, 47, 56, 57]) target: 58
context: tensor([18, 47, 56, 57, 58]) target: 1
context: tensor([18, 47, 56, 57, 58,  1]) target: 15
context: tensor([18, 47, 56, 57, 58,  1, 15]) target: 47
context: tensor([18, 47, 56, 57, 58,  1, 15, 47]) target: 58


In [11]:
batch_size = 4
block_size = 8

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch("train")
print(xb.shape)
print(xb)
print(yb.shape)
print(yb)

print("------")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target: {target}")

torch.Size([4, 8])
tensor([[52, 59, 58, 43,  1, 45, 47, 60],
        [56,  1, 57, 43, 43,  1, 58, 46],
        [49,  1, 63, 53, 59,  1, 52, 53],
        [26, 53, 56, 44, 53, 50, 49,  8]])
torch.Size([4, 8])
tensor([[59, 58, 43,  1, 45, 47, 60, 43],
        [ 1, 57, 43, 43,  1, 58, 46, 43],
        [ 1, 63, 53, 59,  1, 52, 53, 58],
        [53, 56, 44, 53, 50, 49,  8,  0]])
------
when input is [52] the target: 59
when input is [52, 59] the target: 58
when input is [52, 59, 58] the target: 43
when input is [52, 59, 58, 43] the target: 1
when input is [52, 59, 58, 43, 1] the target: 45
when input is [52, 59, 58, 43, 1, 45] the target: 47
when input is [52, 59, 58, 43, 1, 45, 47] the target: 60
when input is [52, 59, 58, 43, 1, 45, 47, 60] the target: 43
when input is [56] the target: 1
when input is [56, 1] the target: 57
when input is [56, 1, 57] the target: 43
when input is [56, 1, 57, 43] the target: 43
when input is [56, 1, 57, 43, 43] the target: 1
when input is [56, 1, 57, 43, 43, 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

n_embd = 32
device = torch.device("cuda")

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each toekn directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.positional_embedding_table = nn.Embedding(block_size, n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
    
    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both (B, T) tensor of integes
        # C is the Channel which represents the embedding table output size
        # when we pass the idx to the token embedding table 
        # we get a embedidng tensor by the idx and get by one by one
        token_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.positional_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = token_emb + pos_emb
        print(x.shape)
        logits = self.lm_head(x) # (B, T, vocab_size)
        print(logits.shape)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # get the prediction
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sample index to the running sequnce
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel().to(device)
out, loss = m(xb.to(device), yb.to(device))
print(out.shape)
print(loss.item())
idx = torch.zeros((1, 1), dtype=torch.long).to(device)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

In [13]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-2)

In [ ]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch("train")

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

In [22]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=400)[0].tolist()))

RuntimeError: ignored

In [23]:
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [24]:
# x[b, t] = mean_{i<=t} x[b, i]
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t, C)
        xbow[b, t] = torch.mean(xprev, 0)

In [25]:
x[0]

tensor([[-0.2871, -0.0677],
        [ 1.2781, -0.2583],
        [-1.0293,  1.2630],
        [-1.3356, -0.4698],
        [ 1.4156,  0.5810],
        [ 0.5460,  0.0431],
        [-1.1980,  0.2230],
        [-0.3582,  0.7448]])

In [26]:
xbow[0]

tensor([[-0.2871, -0.0677],
        [ 0.4955, -0.1630],
        [-0.0128,  0.3124],
        [-0.3435,  0.1168],
        [ 0.0083,  0.2096],
        [ 0.0979,  0.1819],
        [-0.0872,  0.1878],
        [-0.1211,  0.2574]])

In [27]:
# 삼각행렬 
torch.tril(torch.ones(3, 3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [28]:
wei = torch.tril(torch.ones(T, T))
wei = wei/wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) --> (B, T, C)
torch.allclose(xbow2,xbow)

True

In [29]:
# this way may be more efficient than the above one. 
# with trimatrix we can calculate the mean of the context with matrix multiplication
a = torch.ones(3, 3)
a = torch.tril(a)
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b

print(a)
print(b)
print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[9., 3.],
        [4., 4.],
        [8., 9.]])
tensor([[9.0000, 3.0000],
        [6.5000, 3.5000],
        [7.0000, 5.3333]])


In [30]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril==0, float("-inf"))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [31]:
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril==0, float("-inf"))
wei = F.softmax(wei, dim=-1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [32]:
# batch size, time, channel(token_size)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

# query: what im looking for
# key: what do i contain
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
# print((k@q.view(B, head_size, T)).shape)
wei = q @ k.transpose(-2, -1) # careful for the batch dimension
# (B, T, 16) @ (B, 16, T) =====> (B, T, T)  

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril==0, float("-inf"))
wei = F.softmax(wei, dim=-1)
# Without masking => we cannot GPT could learn to generate next token with see them

v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 16])

In [33]:
wei[0]

tensor([[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [7.1698e-01, 2.8302e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [8.5904e-02, 1.5721e-01, 7.5689e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.7680e-01, 1.5268e-01, 3.5254e-01, 3.1798e-01, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [4.8059e-02, 4.8343e-01, 8.4443e-02, 2.0085e-01, 1.8322e-01, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [3.5137e-03, 1.3789e-01, 2.9893e-02, 7.8287e-02, 2.6475e-01, 4.8567e-01,
         0.0000e+00, 0.0000e+00],
        [7.1074e-02, 7.8936e-01, 5.0292e-03, 4.2795e-02, 2.0874e-03, 2.9134e-04,
         8.9363e-02, 0.0000e+00],
        [5.8831e-03, 3.1767e-01, 1.0266e-01, 6.1120e-02, 3.0109e-02, 1.4905e-02,
         2.5145e-01, 2.1621e-01]], grad_fn=<SelectBackward0>)

### Self Attention vs Cross Attention

Self Attention refers that the key and query came from same origin. 
But Cross Attention's key and query came different source like Encoder Vector and Decoder. 

In [36]:
# Why scaled dot product is important
# Because we make Q@K.T to minimize for softmax.
# softmax can converge the values
# we need to initialize to diffuse widely.
# and we use scaled dot product for it
torch.softmax(torch.tensor([0.1, -0.2, 0.3, 0.5])*8, dim=-1), torch.softmax(torch.tensor([0.1, -0.2, 0.3, 0.5]), dim=-1)

(tensor([0.0327, 0.0030, 0.1620, 0.8023]),
 tensor([0.2245, 0.1663, 0.2742, 0.3349]))